In [2]:
!pip install pulp

In [40]:
from pulp import *
from openpyxl import *

#Excelファイル・シートを開く
book = load_workbook('卒研_データ.xlsx')
sheet1 = book['入力値']
sheet2 = book['確認']
sheet3 = book['出力値']

#定数用のデータの作成
I = [i+1 for i in range(9)]
J = [i+1 for i in range(4)]
T = [i+1 for i in range(8)]
n = 0
m = 0

a = {}    
for i in I:
    for t in T:
        a[i, t] = sheet1.cell(row=3+i, column=1+t).value      
    
#空問題の作成
model = LpProblem('Shift', sense=LpMinimize)

#決定変数の作成
x = {}
for i in I:
    for j in J:
        for t in T:
            x[i, j, t] = LpVariable(f'x{i},{j},{t}', cat=LpBinary)
v = {}
for i in I:
    v[i] = LpVariable(f'v{i}', lowBound=0)
w = {}
for i in I:
    for t in T:
        w[i, t] = LpVariable(f'w{i}, {t}', cat=LpBinary)
z = {}
for i in I:
    for t in T:
        z[i, t] = LpVariable(f'z{i}, {t}', cat=LpBinary)

#制約条件の追加

#条件①
for j in J:
    for t in T:
        model += lpSum(x[i, j, t] for i in I) == 1
#条件②
for i in I:
    for t in T:
        model += lpSum(x[i, j, t] for j in J) <= 1
#条件③
for i in I:
    for t in T:
        if a[i, t] == 0:
            for j in J:
                model += x[i, j, t] == 0
#条件⑥
for i in I:
    for t in T:
        if a[i, t] == 2:
            model += lpSum(x[i, j, t] for j in J) == 0 + w[i, t]
        if w[i, 1] == 1:
            m += 1
    sheet2.cell(row=9+i, column=12).value = m
    m = 0
#条件④
s = (len(J) * len(T)) / len(I)
for i in I:
    model += lpSum(x[i, j, t] for j in J for t in T) >= s - v[i]
    model += lpSum(x[i, j, t] for j in J for t in T) <= s + v[i]
#条件⑤
for i in I:
    for t in range(len(T)-1):
        model += lpSum(x[i, j, t+1] for j in J) + lpSum(x[i, j, t+2] for j in J) <= 1 + z[i, t+1]
        
#目的関数の設定
model +=lpSum(v[i] for i in I) + lpSum(w[i, t] for i in I for t in T) + lpSum(z[i, t] for i in I for t in T)

#最適化の実行
model.solve()
print(v)
print(w)
print(z)

#最適化の結果出力
if LpStatus[model.status] == 'Optimal':
    print('最適値 =', value(model.objective))
    I_Name = {1:'西村', 2:'松野', 3:'大屋', 4:'牛尾', 5:'田中', 6:'押見', 7:'出口', 8:'伊藤', 9:'藤木'}
    for i in I:
        for j in J:
            for t in T:
                if value(x[i, j, t]) > 0.01:
                    sheet2.cell(row=2+j, column=1+t).value = I_Name[i]    #出力値に名前を出力
                    sheet3.cell(row=2+j, column=1+t).value = I_Name[i]    #シフト表に名前を入力
    print(z[i, t])   
   # for i in I:
    #    for t in T:
     #       if z[i, t] == 1:
      #          n += 1
       #     #print(z[i, t])
        #sheet2.cell(row=9+i, column=11).value = n
        #n = 0
    book.save('卒研_データ_1.xlsx')
else:
    print('最適解が求まりませんでした。')

{1: v1, 2: v2, 3: v3, 4: v4, 5: v5, 6: v6, 7: v7, 8: v8, 9: v9}
{(1, 1): w1,_1, (1, 2): w1,_2, (1, 3): w1,_3, (1, 4): w1,_4, (1, 5): w1,_5, (1, 6): w1,_6, (1, 7): w1,_7, (1, 8): w1,_8, (2, 1): w2,_1, (2, 2): w2,_2, (2, 3): w2,_3, (2, 4): w2,_4, (2, 5): w2,_5, (2, 6): w2,_6, (2, 7): w2,_7, (2, 8): w2,_8, (3, 1): w3,_1, (3, 2): w3,_2, (3, 3): w3,_3, (3, 4): w3,_4, (3, 5): w3,_5, (3, 6): w3,_6, (3, 7): w3,_7, (3, 8): w3,_8, (4, 1): w4,_1, (4, 2): w4,_2, (4, 3): w4,_3, (4, 4): w4,_4, (4, 5): w4,_5, (4, 6): w4,_6, (4, 7): w4,_7, (4, 8): w4,_8, (5, 1): w5,_1, (5, 2): w5,_2, (5, 3): w5,_3, (5, 4): w5,_4, (5, 5): w5,_5, (5, 6): w5,_6, (5, 7): w5,_7, (5, 8): w5,_8, (6, 1): w6,_1, (6, 2): w6,_2, (6, 3): w6,_3, (6, 4): w6,_4, (6, 5): w6,_5, (6, 6): w6,_6, (6, 7): w6,_7, (6, 8): w6,_8, (7, 1): w7,_1, (7, 2): w7,_2, (7, 3): w7,_3, (7, 4): w7,_4, (7, 5): w7,_5, (7, 6): w7,_6, (7, 7): w7,_7, (7, 8): w7,_8, (8, 1): w8,_1, (8, 2): w8,_2, (8, 3): w8,_3, (8, 4): w8,_4, (8, 5): w8,_5, (8, 6): w8,_6, (8, 7